# Fixed Coupon Bond

The goal of the project is to compute the market value of a portfolio of fixed coupon bonds taking into account the issuer credit risk. The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* A set of survival probabilities and the recovery rate of the issuer
* The static data of the pool of bonds (nominal, start date, end date, coupons, payment frequency)

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

In [2]:
from datetime import date

In [3]:
today = date(2019, 10, 31)

In [4]:
ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

In [6]:
survival_probabilities = [
    {'date': date(2020, 12, 20), 'ndp': 0.972159727015014},
    {'date': date(2021, 12, 20), 'ndp': 0.942926329174406},
    {'date': date(2022, 12, 20), 'ndp': 0.913448056250137},
    {'date': date(2024, 12, 20), 'ndp': 0.855640452819766},
    {'date': date(2029, 12, 20), 'ndp': 0.732687779675469},
    {'date': date(2039, 12, 20), 'ndp': 0.539046016487758},
]

In [8]:
bonds_to_price = [
    {'nominal': 4972284.02, 'start_date': date(2016, 3, 1), 'end_date': date(2027, 8, 1),
     'coupon_frequency': 3, 'coupon': 0.035, 'recovery': 0.2},
    {'nominal': 7344328.27, 'start_date': date(2015, 7, 1), 'end_date': date(2022, 7, 1), 
     'coupon_frequency': 3, 'coupon': 0.035, 'recovery': 0.2},
    {'nominal': 7172290.19, 'start_date': date(2019, 1, 1), 'end_date': date(2023, 9, 1), 
     'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 7065224.23, 'start_date': date(2016, 3, 1), 'end_date': date(2020, 10, 1), 
     'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 5256452.14, 'start_date': date(2017, 7, 1), 'end_date': date(2022, 4, 1), 
     'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 2689680.89, 'start_date': date(2015, 9, 1), 'end_date': date(2030, 7, 1), 
     'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 3593518.71, 'start_date': date(2016, 7, 1), 'end_date': date(2025, 2, 1), 
     'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6993589.53, 'start_date': date(2017, 1, 1), 'end_date': date(2024, 11, 1), 
     'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6684377.52, 'start_date': date(2015, 9, 1), 'end_date': date(2027, 9, 1), 
     'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6896199.04, 'start_date': date(2016, 7, 1), 'end_date': date(2024, 11, 1), 
     'coupon_frequency': 12, 'coupon': 0.027, 'recovery': 0.4},
    {'nominal': 2587984.6, 'start_date': date(2017, 10, 1), 'end_date': date(2026, 10, 1), 
     'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 3621656.1, 'start_date': date(2018, 6, 1), 'end_date': date(2022, 7, 1), 
     'coupon_frequency': 6, 'coupon': 0.027, 'recovery': 0.4},
    {'nominal': 3146567.47, 'start_date': date(2017, 6, 1), 'end_date': date(2028, 3, 1), 
     'coupon_frequency': 6, 'coupon': 0.018, 'recovery': 0.2},
    {'nominal': 6452721.61, 'start_date': date(2015, 4, 1), 'end_date': date(2025, 4, 1), 
     'coupon_frequency': 3, 'coupon': 0.018, 'recovery': 0.2},
    {'nominal': 3418346.24, 'start_date': date(2016, 5, 1), 'end_date': date(2022, 1, 1), 
     'coupon_frequency': 3, 'coupon': 0.018, 'recovery': 0.2},
]